In [0]:
import tensorflow as tf

def create_placeholder(n_H0, n_W0, n_C0, n_Y):
  
  X = tf.placeholder(tf.float32, [None, n_H0, n_W0, n_C0], name = 'X')
  Y = tf.placeholder(tf.float32, [None, n_Y], name = 'Y')
  
  return X, Y

def initialize_parameter():
  
  parameters = {}
  
  W1 = tf.get_varaible('W1', [4, 4, 3, 8], initializer = tf.contrib.layers.xavier_initializer())
  W2 = tf.get_varaible('W2', [2, 2, 8, 16],initializer = tf.contrib.layers.xavier_initializer())

  parameters = {
                'W1' : W1,
                'W2' : W2
               }
  
  return W1, W2

def conv_forward_propagation(X, parameters):
  
  W1 = parameters['W1']
  W2 = parameters['W2']
  
  Z1 = tf.nn.conv2d(X, W1, stride = [1, 1, 1 ,1], padding = 'SAME')
  A1 = tf.nn.relu(Z1)
  P1 = tf.nn.max_pool(A1, ksize = [1, 8, 8, 1], stride = [1, 8, 8, 1], padding = 'SAME')
  
  Z2 = tf.nn.conv2d(P1, W2, stride = [1, 1, 1 ,1], padding = 'SAME')
  A2 = tf.nn.relu(Z2)
  P2 = tf.nn.max_pool(A2, ksize = [1, 4, 4, 1], stride = [1, 4, 4, 1], padding = 'SAME')
  
  P2 = tf.contrib.layers.flatten(P2)
  Z3 = tf.contrib.layers.fully_connected(P2, 6, activation_fn = None)
  
  return Z3
  
def compute_cost(Z3, Y):
  
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y))
  
  return cost

def nn_model(X_train, Y_train, X_test, Y_test, minibatch_size, num_epochs, learning_rate = .009 print_cost = True):
  
  m, n_H0, n_W0, n_C0 = X_train.shape
  n_Y = Y_train.shape[1]
  
  X, Y = create_placeholder(n_H0, n_W0, n_C0, n_Y)
  
  parameters = initialize_parameter()
  Z3 = conv_forward_propagation(X, parameters)
  
  cost = compute_cost(Z3, Y)
  
  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
  
  init = tf.global_variables_initializer()
  
  with tf.Session() as sess:
    sess.run(init)
    
    minibatch_cost = 0
    num_minibatches = int(m / minibatch_size)
    
    minibatches = random_mini_batch(X_train, y_train, minibatch_size)
    
    for i in range(num_epochs):
      
      for minibatch in minibatches:

        (minibatch_X, minibatch_Y) = minibatch
        _, _, minibatch_cost = sess.run([optimizer, cost], feed_dict = {X : minibatch_X, Y : minibatch_Y})

        total_cost += minibatch_cost / num_minibatches

        
    correct_prediction = tf.equal(tf.argmax(Z3, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    train_accuracy = accuracy.eval({X : X_train, Y : Y_train})
    test_accuracy = accuracy.eval({X : X_test, Y : Y_test})
    
    return train_accuracy, test_accuracy
    
       